In [1]:
# To use the files in the parent directory run this cell
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.models.vqvae import VQVAE
from src.models.var import VAR
from src.datasets.hugging_face_dataset import HuggingFaceDataset

In [3]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import gc


model_params = {
    "mnist": {
        "VQVAE_DIM": 64,
        "VOCAB_SIZE": 32,
        "PATCH_SIZES": [1, 2, 3, 4, 8],
        "VAR_DIM": 64,
        "N_HEADS": 4,
        "N_LAYERS": 6,
        "channels": 1,
    },
    "cifar": {
        "VQVAE_DIM": 512,
        "VOCAB_SIZE": 2048,
        "PATCH_SIZES": [1, 2, 3, 4, 6, 8],
        "VAR_DIM": 512,
        "N_HEADS": 16,
        "N_LAYERS": 12,
        "channels": 3,
    },
    "small": {
        "VQVAE_DIM": 512,
        "VOCAB_SIZE": 1024,
        "PATCH_SIZES": [1, 2, 3, 4, 6, 8],
        "VAR_DIM": 512,
        "N_HEADS": 16,
        "N_LAYERS": 16,
        "channels": 3,
    },
    "medium": {
        "VQVAE_DIM": 512,
        "VOCAB_SIZE": 2048,
        "PATCH_SIZES": [1, 2, 3, 4, 6, 8],
        "VAR_DIM": 512,
        "N_HEADS": 32,
        "N_LAYERS": 20,
        "channels": 3,
    },
    "large": {
        "VQVAE_DIM": 512,
        "VOCAB_SIZE": 4096,
        "PATCH_SIZES": [1, 2, 3, 4, 6, 8],
        "VAR_DIM": 512,
        "N_HEADS": 64,
        "N_LAYERS": 24,
        "channels": 3,
    },
}

training_params = {
    "mnist": {
        "VQVAE": {
            "batch_size": 2048,
            "lr": 3e-4,
            "epochs": 40,
        },
        "VAR": {
            "batch_size": 1024,
            "lr": 1e-3,
            "epochs": 100,
        },
    },
    "cifar": {
        "VQVAE": {
            "batch_size": 512,
            "lr": 3e-4,
            "epochs": 100,
        },
        "VAR": {
            "batch_size": 64,
            "lr": 1e-4,
            "epochs": 100,
        },
    },
}


def get_data(batch_size=1024, dataset="mnist"):
    if dataset == "cifar":
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.RandomCrop(32),
                transforms.RandomHorizontalFlip(),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )
        train_ds = datasets.CIFAR10(
            root="./data", train=True, download=True, transform=transform
        )
        test_ds = datasets.CIFAR10(
            root="./data", train=False, download=True, transform=transform
        )
    elif dataset == "mnist":
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Pad(2),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )
        train_ds = datasets.MNIST(
            root="./data", train=True, download=True, transform=transform
        )
        test_ds = datasets.MNIST(
            root="./data", train=False, download=True, transform=transform
        )
    else:
        transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Resize((32, 32)),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )
        # Use HuggingFace datasets
        train_ds = HuggingFaceDataset(dataset_path=dataset, split="train", transform=transform)
        test_ds = HuggingFaceDataset(dataset_path=dataset, split="val", transform=transform)


    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=False, drop_last=False
    )
    test_loader = DataLoader(
        test_ds, batch_size=batch_size, shuffle=False, drop_last=True
    )

    print(len(train_loader), len(test_loader))

    return train_loader, test_loader


def plot_images(pred, original=None):
    n = pred.size(0)
    pred = pred * 0.5 + 0.5
    pred = pred.clamp(0, 1)
    img = pred.cpu().detach()

    if original is not None:
        original = original * 0.5 + 0.5
        original = original.clamp(0, 1)
        original = original.cpu().detach()
        img = torch.cat([original, img], dim=0)

    img_grid = make_grid(img, nrow=n)
    img_grid = img_grid.permute(1, 2, 0).numpy()
    img_grid = (img_grid * 255).astype("uint8")
    plt.imshow(img_grid)
    plt.axis("off")


In [4]:
dataset = "zzsi/afhq64_16k"
model_params = model_params["small"]
training_params = training_params["cifar"]

# Training VQVAE

In [4]:
from tqdm import tqdm

print("=" * 10 + "Training VQVAE" + "=" * 10)
vq_model = VQVAE(
    model_params["VQVAE_DIM"],
    model_params["VOCAB_SIZE"],
    model_params["PATCH_SIZES"],
    num_channels=model_params["channels"],
)
optimizer = torch.optim.AdamW(
    vq_model.parameters(), lr=training_params["VQVAE"]["lr"]
)

train_loader, test_loader = get_data(
    batch_size=training_params["VQVAE"]["batch_size"], dataset=dataset
)
vq_model = vq_model.to("cuda")

# All epochs
for epoch in tqdm(range(training_params["VQVAE"]["epochs"])):
    epoch_loss = 0
    epoch_recon_loss = 0
    # Single epochs
    for i, (x, c) in enumerate(train_loader):
        x, c = x.cuda(), c.cuda()
        optimizer.zero_grad()
        xhat, r_maps, idxs, scales, q_loss = vq_model(x)
        recon_loss = F.mse_loss(xhat, x)
        loss = recon_loss + q_loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_recon_loss += recon_loss.item()

    epoch_loss /= len(train_loader)
    epoch_recon_loss /= len(train_loader)
    print(f"Epoch: {epoch}, Loss: {epoch_loss}, Recon Loss: {epoch_recon_loss}")

    if epoch % 5 == 0:
        with torch.no_grad():
            total_loss = 0
            total_recon_loss = 0
            for i, (x, c) in enumerate(test_loader):
                x, c = x.cuda(), c.cuda()
                xhat, r_maps, idxs, scales, q_loss = vq_model(x)
                recon_loss = F.mse_loss(xhat, x)
                loss = recon_loss + q_loss
                total_loss += loss.item()
                total_recon_loss += recon_loss.item()

            total_loss /= len(test_loader)
            total_recon_loss /= len(test_loader)

            print(
                f"Epoch: {epoch}, Test Loss: {total_loss}, Test Recon Loss: {total_recon_loss}"
            )

            x = x[:10, :].cuda()
            x_hat = vq_model(x)[0]

            plot_images(pred=x_hat, original=x)
            plt.savefig(f"vqvae_{epoch}.png")
            plt.close()

torch.save(vq_model.state_dict(), "vqvae.pth")
del vq_model, optimizer, x, x_hat, train_loader, test_loader
gc.collect()
torch.cuda.empty_cache()


==========Training VQVAE==========
29 2


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0, Loss: 4.866057824471901, Recon Loss: 0.2037651019877401


  2%|▏         | 1/50 [02:15<1:50:34, 135.40s/it]

Epoch: 0, Test Loss: 3.8543347120285034, Test Recon Loss: 0.2917487621307373


...

Epoch: 47, Loss: 0.06947716338367298, Recon Loss: 0.01871033449625147


 98%|█████████▊| 49/50 [1:49:12<02:13, 133.68s/it]

Epoch: 48, Loss: 0.06802729388763165, Recon Loss: 0.018576490981825466


100%|██████████| 50/50 [1:51:25<00:00, 133.72s/it]

Epoch: 49, Loss: 0.06796855762087066, Recon Loss: 0.0177432169238555


# Training VAR

In [5]:
with torch.no_grad():
    torch.cuda.empty_cache()

In [6]:
print("=" * 10 + "Training VAR" + "=" * 10)
vqvae = VQVAE(
    model_params["VQVAE_DIM"],
    model_params["VOCAB_SIZE"],
    model_params["PATCH_SIZES"],
    num_channels=model_params["channels"],
)
vqvae.load_state_dict(torch.load("small_vqvae.pth"))  # LOADS the trained VQVAE model
vqvae = vqvae.to("cuda")
vqvae.eval()

for param in vqvae.parameters():
    param.requires_grad = False

var_model = VAR(
    vqvae=vqvae,
    dim=model_params["VAR_DIM"],
    n_heads=model_params["N_HEADS"],
    n_layers=model_params["N_LAYERS"],
    patch_sizes=model_params["PATCH_SIZES"],
    n_classes=10,
)
optimizer = torch.optim.AdamW(
    var_model.parameters(), lr=training_params["VAR"]["lr"]
)

print(f"VQVAE Parameters: {sum(p.numel() for p in vqvae.parameters())/1e6:.2f}M")
print(f"VAR Parameters: {sum(p.numel() for p in var_model.parameters())/1e6:.2f}M")

train_loader, test_loader = get_data(
    batch_size=training_params["VAR"]["batch_size"], dataset=dataset
)
var_model = var_model.to("cuda")
for epoch in range(training_params["VAR"]["epochs"]):
    epoch_loss = 0
    for i, (x, c) in enumerate(train_loader):
        x, c = x.cuda(), c.cuda()
        optimizer.zero_grad()

        _, _, idxs_R_BL, scales_BlC, _ = vqvae(x)
        idx_BL = torch.cat(idxs_R_BL, dim=1)
        scales_BlC = scales_BlC.cuda()
        logits_BLV = var_model(scales_BlC, cond=c)
        loss = F.cross_entropy(
            logits_BLV.view(-1, logits_BLV.size(-1)), idx_BL.view(-1)
        )

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(train_loader)
    print(f"Epoch: {epoch}, Loss: {epoch_loss}")

    if epoch % 5 == 0:
        with torch.no_grad():

            cond = torch.arange(10).cuda()
            out_B3HW = var_model.generate(cond, 0)
            plot_images(pred=out_B3HW)

            plt.savefig(f"var_{epoch}.png")
            plt.close()

torch.save(var_model.state_dict(), "var.pth")

==========Training VAR==========


/tmp/ipykernel_6209/1893514572.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vqvae.load_state_dict(torch.load("small_vqvae.pth"))  # LOADS the trained VQVAE model


VQVAE Parameters: 85.00M
VAR Parameters: 178.97M
229 23
Epoch: 0, Loss: 4.210374780096862
Epoch: 1, Loss: 4.026469395150264
Epoch: 2, Loss: 3.9341810374280772
Epoch: 3, Loss: 3.851914644241333
Epoch: 4, Loss: 3.745109129160252
Epoch: 5, Loss: 3.6351813031075824
Epoch: 6, Loss: 3.5295720475209333
Epoch: 7, Loss: 3.427367805914067
Epoch: 8, Loss: 3.3203730312497335
Epoch: 9, Loss: 3.2066679937870743
Epoch: 10, Loss: 3.081625181514623
Epoch: 11, Loss: 2.9439822003310425
Epoch: 12, Loss: 2.7930768155635186
Epoch: 13, Loss: 2.6147650490681675
Epoch: 14, Loss: 2.43959974930276
Epoch: 15, Loss: 2.241192906704532
Epoch: 16, Loss: 2.0920631901145503
Epoch: 17, Loss: 1.9529445371773566
Epoch: 18, Loss: 1.8152784655708414
Epoch: 19, Loss: 1.6868888712345773
Epoch: 20, Loss: 1.5457673895306983
Epoch: 21, Loss: 1.4383269316764897
Epoch: 22, Loss: 1.3347437722714186
Epoch: 23, Loss: 1.2131889936184779
Epoch: 24, Loss: 1.1350385648194359
Epoch: 25, Loss: 1.047056037265661
Epoch: 26, Loss: 0.966550968